In [1]:
#import packages & dependencies

from packages import *
from models import Generator, Discriminator, define_autoencoder, define_discriminator, define_generator
from train_autoencoder import train_autoencoder
from train import train
from stats import simulate, moment_dim_wise, plot_dim_mo, show, get_p_values, get_kl_norms, get_stats_save_models

def plot_model(m, **kwargs):
    return tf.keras.utils.plot_model(
        m, show_shapes=True, show_layer_activations=True, show_layer_names= True, expand_nested= True, **kwargs
    )

In [2]:
#import source data

pi = pd.read_pickle('prepared_data_tabular_data_v1403.pkl')
cols = pi.columns
data = np.array(pi)
del pi
whole_data = tf.constant(data)
whole_data.shape

TensorShape([12651, 1742])

In [5]:
#hyperparameters

inputDim = whole_data.shape[-1]
embeddingDim = 128
randomDim = 128
aeActivation = 'relu'
discriminatorActivation = 'relu'
dataType = ''
generatorDims = (128,128)
discriminatorDims = (256,128,1)
keepRate = 1.0
is_trainable = True
lr = 0.001

In [6]:
# autoencoder pretraining

ae, enc, dec = define_autoencoder(inputDim, embeddingDim, aeActivation, randomDim, dataType='')
history = train_autoencoder(ae, whole_data, epochs=100, lr=0.001)

Processing pretraining. Current mean batch loss : 0: 100%|█| 1/1 [00:06<00:00, 


In [7]:
# gan & dec training

g,d = (define_generator(randomDim, generatorDims, is_trainable, dec, dataType=''),
        define_discriminator(inputDim, discriminatorDims, discriminatorActivation, keepRate))
lgen, ldis, g, d = train(data_seq=whole_data.numpy(), gen_model= g, dis_model= d, latent_dim=embeddingDim, epochs=1000, batch_size=32, lr=0.001)

100%|████████████████████████████████████████████| 2/2 [00:12<00:00,  6.46s/it]


In [8]:
#Generating & saving data after model training

gen = np.rint(g.call(tf.random.normal((whole_data.shape[0],128))).numpy()).astype(int)
print(gen.shape)
np.save('simulated_data.npy', gen)

(12651, 1742)